In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
url = "https://labor.ny.gov/app/warn/default.asp?warnYr=2017"
url

'https://labor.ny.gov/app/warn/default.asp?warnYr=2017'

In [5]:
headers = {'accept-encoding': 'deflate'}
headers

{'accept-encoding': 'deflate'}

In [6]:
response = requests.get(url, headers=headers)
response

<Response [200]>

In [7]:
response.status_code

200

In [8]:
type(response.text)

str

In [54]:
soup = BeautifulSoup (response.text, 'html.parser')

In [55]:
table = soup.find("table")

In [56]:
rows =soup.find_all("tr")

In [60]:
len(rows)

321

In [61]:
# indicate the url start of the individual case URLs as they aren't the same as the launching page
newurl = "https://labor.ny.gov/app/warn/"

In [65]:
results = []
for row in rows:
    #print(row)
    a = row.find("a")['href']
    #print(a)
    company_url = f'{newurl}{a}'
    print(company_url)
    company_response = requests.get(company_url, headers=headers)
   # print (company_response.status_code)
    company_soup = BeautifulSoup(company_response.text, 'html.parser') 
   # print(company_soup)
    company_table = company_soup.find("table")
    # print(company_table)
    paragraphs = company_table.find_all("p")
    for p in paragraphs: 
        text=p.get_text('\n')
        if ('Date of Notice:' in text):
            notice_date = text.split(":")[1].strip().split('\n')[0].split()[0].replace(',', '').replace(';', '')
           # print(notice_date)
        elif ('Reason Stated for Filing:' in text):
            reason = text.split(":")[1].strip()
           # print(reason)
        elif ('Company:' in text):
            split_company = text.split('\n')
            company = split_company[1].strip()
            address = " ".join(split_company[2:])
            #print(company)
            #print(address)
        elif ('Phone:' in text):
            phone = text.split(":")[1].strip()
            #print(phone)
        elif ('Business Type:' in text):
            business_type = text.split(":")[1].strip()
            #print(business_type)
        elif ('Number Affected:' in text):
            affected = text.split(":")[1].strip().split()[0].replace(',', '')
            #print(affected)
        elif ('Total Employees:' in text):
            total_employees = text.split(":")[1].strip()
            #print(total_employees)
        elif ('Layoff Date:' in text):
            layoff_date = text.split(":")[1].strip()
            #print(layoff_date)
        elif ('Reason for Dislocation:' in text):
            dislocation = text.split(":")[1].strip()
            #print(dislocation)
        elif ('Union:' in text):
            union = text.split(":")[1].strip()
            #print(union)
        elif ('Classification:' in text):
            classification = text.split(":")[1].strip()
            #print(classification)
    result = {
        'notice_date': notice_date,
        'reason': reason,
        'company': company,
        'address': address,
        'phone': phone,
        'business_type': business_type,
        'affected': affected,
        'total_employees': total_employees,
        'layoff_date': layoff_date,
        'dislocation': dislocation,
        'union': union,
        'classification': classification
     }       
    results.append(result) 


https://labor.ny.gov/app/warn/details.asp?id=6166
https://labor.ny.gov/app/warn/details.asp?id=6167
https://labor.ny.gov/app/warn/details.asp?id=6168
https://labor.ny.gov/app/warn/details.asp?id=6164
https://labor.ny.gov/app/warn/details.asp?id=6165
https://labor.ny.gov/app/warn/details.asp?id=6162
https://labor.ny.gov/app/warn/details.asp?id=6163
https://labor.ny.gov/app/warn/details.asp?id=6160
https://labor.ny.gov/app/warn/details.asp?id=6161
https://labor.ny.gov/app/warn/details.asp?id=6155
https://labor.ny.gov/app/warn/details.asp?id=6156
https://labor.ny.gov/app/warn/details.asp?id=6157
https://labor.ny.gov/app/warn/details.asp?id=6158
https://labor.ny.gov/app/warn/details.asp?id=6159
https://labor.ny.gov/app/warn/details.asp?id=6154
https://labor.ny.gov/app/warn/details.asp?id=6040
https://labor.ny.gov/app/warn/details.asp?id=6153
https://labor.ny.gov/app/warn/details.asp?id=6149
https://labor.ny.gov/app/warn/details.asp?id=6150
https://labor.ny.gov/app/warn/details.asp?id=6151


In [66]:
df= pd.DataFrame(results)
df.head()

,notice_date,reason,company,address,phone,business_type,affected,total_employees,layoff_date,dislocation,union,classification
0,12/21/2017,Plant Layoff,Eastman Kodak Company (Eastman Business Park),"1669 Lake Avenue Rochester, NY 14650",(585) 724-7080,Photographic equipment and supplies,2,-----,3/20/2018,Economic,The affected workers are not represented by a ...,Plant Layoff
1,9/11/2017,Plant Layoff,"DynaServ Industries, Inc.","5830 64th Street Maspeth, NY 11378",(718) 416-0252 Ext,Specialized Services,77,122,All employee separations are expected to occur...,Loss of client contract,IBEW Local 3,Plant Layoff
2,6/28/2017,Plant Closing,Courtyard by Marriott LaGuardia Hotel,"90-10 Grand Central Parkway East Elmhurst, NY ...",(718) 308-4803,Hotel,140,140,The closure of the hotel and permanent layoff ...,Economic,"New York Hotel & Motel Trades Council, AFL-CIO",Plant Closing
3,12/22/2017,Plant Closing,"CareConnect Insurance Company, Inc.","2200 Northern Boulevard East Hills, NY 11548",(516) 405-7925,Provide health insurance,42,218,Phase II separations will begin on or about 1/...,Economic,The affected workers are not represented by a ...,Plant Closing
4,9/13/2017,Plant Layoff,"Sykes Enterprises, Incorporated","251 John James Audubon Pkwy Amherst, NY 14228",(701) 425-7794,Customer contact management solutions and serv...,36,131,Layoffs extended from 12/29/2017 to 3/31/2018.,Economic,The affected workers are not represented by a ...,Plant Layoff


In [68]:
df.to_csv('../data/2017.csv', index=False)